In [ ]:
import matplotlib
from datascience import *
%matplotlib inline
import matplotlib.pyplot as plots
import numpy as np
plots.style.use('fivethirtyeight')

# Activity: Bootstrapping Confidence Intervals

The cell below will load in a dataset from Inside Airbnb (http://insideairbnb.com/get-the-data.html) that has details of over 2000 Airbnb listings in Asheville, NC through September 2023. It includes information about the name of the property, the zip code it's in, price, and many other fields.

Today, we'll be investigating how we might be able to estimate the average price for a night at an Airbnb in Asheville, NC. There was one property with a an incorrect price listed at over $10,000, which will be removed from the dataset.

In [ ]:
asheville = Table.read_table('data/asheville-airbnb-2023.csv').where('price', are.below(10000))
asheville

## Important Note:
Throughout this activity, we are going to practice our bootstrapping technique to construct a confidence interval from a single sample from the population. **Remember:** We use the bootstrap technique when we don't have access to a population, only a sample from that population. That means for this activity, **pretend** that you don't have access to the full population unless specifically asked to use it.

## Draw a Sample

Remember, our goal is to estimate the average nightly price of all Airbnb's in Asheville, NC. As discussed in the lessons, the best way to make such an estimate is to construct a confidence interval. As we've seen, larger sample sizes produce more narrow confidence intervals, meaning we have a more precise estimate of our value. However, larger samples are more expensive to take in real life (you have to spend either more time or more money to get more data in your sample). There's an inherent trade off between cost and accuracy, and when solving your own problems you have to decide how much the additional accuracy is worth to you! In a future lesson you'll learn a method for determining what size sample you should take in order to achieve a certain level of accuracy, but for now choose a sample size between 50 and 200 to use and assign it to the variable `sample_size` in the code cell below.

Then, create a sample from the `asheville` table of size `sample_size` and assign that Table to `airbnb_sample`.

In [ ]:
sample_size = 250
airbnb_sample = asheville.sample(sample_size)

# Show the top 5 to see a little about your sample
airbnb_sample.show(5)

# Bootstrapping the Sample

Since this is the only sample we have to use, write a function called `one_bootstrap_mean` that takes in no arguments and returns the mean value of the `price` column of a single resample of `airbnb_sample`. **Remember:** When resampling (taking a bootstrap) from a sample, you should draw the same number of elements that are contained in the sample (`sample_size`) and you should draw **with replacement**!

In [ ]:
def one_bootstrap_mean():
    return np.mean(airbnb_sample.sample(sample_size, with_replacement=True).column('price'))

one_bootstrap_mean()

Now, create an array named `bootstrap_means` which contains 1000 bootstrap means of your `airbnb_sample` Table. You should follow our typical format of first constructing an empty array and then writing a `for` loop that computes the bootstrap mean (use your function from above) and appends each mean to the `bootstrap_means` array.

In [ ]:
bootstrap_means = make_array()

for i in np.arange(1000):
    new_mean = one_bootstrap_mean()
    bootstrap_means = np.append(bootstrap_means, new_mean)

# Shows ONLY the first 10 bootstrap means
bootstrap_means[0:10]

Create a histogram of these bootstrap means. Theory predicts that you should expect to see a normal distribution of the bootstrap means. Do you?

**Hint:** Remember, you can quickly create a histogram from an array by constructing an empty Table using `Table()` and then appending the array as a column to the table by using `.with_column( ... )` and then calling `.hist( ... )`. Be sure to provide all the needed arguments to these methods. 

In [ ]:
Table().with_column('Bootstrap means', bootstrap_means).hist()
plots.title('Bootstrap Means (1K Bootstraps from our Sample)');

In [ ]:
min(bootstrap_means), max(bootstrap_means)

## Constructing the Confidence Interval

To construct a 95% Confidence Interval you'll need to determine which two values in the bootstrapped distribution contain 95% of the data between them.

Use the `percentile` function to determine the left and right endpoints of a 95% Confidence Interval for the mean price. Assign these values to `left` and `right` respectively.

In [ ]:
left = percentile(2.5, bootstrap_means)
right = percentile(97.5, bootstrap_means)

print("95% CI: (", left, ",", right,")")

The code below will overlay a yellow line across the base of the distribution of bootstrapped means to visually show the values in the distribution that are within the interval.

In [ ]:
Table().with_column('Bootstrap means', bootstrap_means).hist()
plots.plot([left,right], [0,0], color="gold", lw=8, zorder=1);
plots.title('Bootstrap Means (1K Bootstraps from our Sample)');

## How did you do?

The goal of constructing this 95% Confidence Interval was to estimate the true average price for a night in an Airbnb in Asheville, NC. Unless we had the full dataset there'd be no way to know this actual value. In this case, we **do** have the full dataset, so we can see if our sample and bootstrapping technique resulted in an interval that contains the true population average. We should expect this process to construct an interval that contains the true mean 95% of the time, so if your interval doesn't contain the mean that doesn't necessarily mean you did this wrong! Let's see how you did.

First, let's look at the distribution of ALL prices:

In [ ]:
asheville.hist('price', bins = np.arange(0,1000,50))

Next, let's look at what the average of the price column is for the `asheville` table.

In [ ]:
population_mean = np.mean(asheville.column('price'))
population_mean

Is the population mean in between `left` and `right`, or put another way, did your Confidence Interval contain the true mean?

In [ ]:
left < population_mean < right

The code below will recreate this entire process 20 times and draw all 20 confidence intervals as horizontal yellow lines. Notice how variations in the original sample that was drawn, as well as randomness in the resampling process, create variations in the Confidence Intervals. In real life, you're usually only able to construct 1 of these from your single sample you were able to obtain. You'd never know the details of how other samples and other resamples might have impacted how it *could* have gone differently. Do any of these Confidence Intervals fail to capture the true average that we were trying to predict? The true average price is drawn in as a vertical red line. If any of your yellow bars don't contain the red line, that means if you had constructed that as your confidence interval it would have NOT been a good estimate for the mean price. Because we constructed a 95% confidence interval, we'd expect that to happen about 5% of the time, or 1 out of every 20 Confidence Intervals.

**Caution:** This next cell might take a little while to run!

In [ ]:
for i in np.arange(20):
    airbnb_sample = asheville.sample(250)
    
    bootstrap_means = make_array()
    for j in np.arange(1000):
        new_mean = one_bootstrap_mean()
        bootstrap_means = np.append(bootstrap_means, new_mean)
        left = percentile(2.5, bootstrap_means)
        right = percentile(97.5, bootstrap_means)
        plots.plot([left,right], [i,i], color="gold", lw=8, zorder=1);
    plots.plot([population_mean,population_mean], [-1,i+1], color="red", lw=1, zorder=1);
    plots.title('20 Confidence Intervals');